In [5]:
from src.data_processing.fetch_data import fetch_data_subgraph
from src.models.liquidity_distribution import Pool, LiquidityPeriphery

%load_ext autoreload
%autoreload 2


API_KEY = "893d7471304c5edf436c8ba60781762c"
POOL_ADDRESS = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640".lower()
pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    totalValueLockedUSD
    totalValueLockedETH
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""
# POOL_ADDRESS = "0xc7bBeC68d12a0d1830360F8Ec58fA599bA1b0e9b".lower()
variables = {'pool_id': POOL_ADDRESS}
data_pool = fetch_data_subgraph(API_KEY, pool_query, variables, data_key='pools', first_n=100, batch_size=1000)[0]
pool = Pool(pool_address=POOL_ADDRESS,
            token0=data_pool['token0']['symbol'],
            token1=data_pool['token1']['symbol'],
            decimals0=data_pool['token0']['decimals'],
            decimals1=data_pool['token1']['decimals'],
            fee_tier=data_pool['feeTier'],
            sqrt_price_x96=data_pool["sqrtPrice"])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Fetched 1 records


### Example calculation from ticks 

In [3]:
pool.token0, pool.token1

('USDC', 'WETH')

In [6]:
lower_tick = 200070
upper_tick = 202250
current_tick = 201101

initial_capital_amount_0 = 1000 #USDC
initial_capital_amount_1 = .5409 #WETH

First we initialize class the functions to convert ticks to sqrt of price in q96 which is used for precision. 

In [7]:
liquidity_calculator = LiquidityPeriphery(pool)

We can can calcualte sqrt of price from tick, this is the implementation of getSqrtRatioAtTick function from Tickmath: https://github.com/Uniswap/v3-core/blob/main/contracts/libraries/TickMath.sol

In [8]:
sqrt_ratio_x96 = liquidity_calculator.get_sqrt_ratio_at_tick(current_tick)
sqrt_ratio_a_x96 = liquidity_calculator.get_sqrt_ratio_at_tick(lower_tick)
sqrt_ratio_b_x96 = liquidity_calculator.get_sqrt_ratio_at_tick(upper_tick)

Once we have that we also need amounts of token0 and token1. However, we need to convert these to wei. 

In [12]:
amount0wei: int = pool.to_wei(initial_capital_amount_0, pool.decimals0)

In [19]:
amount0wei

1000000000

In [14]:
amount1wei: int = pool.to_wei(initial_capital_amount_1, pool.decimals1)

In [20]:
amount1wei

540900000000000064

In [23]:
_, _, _, _, _, liquidity = liquidity_calculator.get_liquidity_for_amounts(sqrt_ratio_x96, sqrt_ratio_a_x96, sqrt_ratio_b_x96, amount0wei, amount1wei)

In [25]:
The liquidity we provide can be found below, this is a single value that we provide during the whole duration when the current price is within our defined LP range. 

SyntaxError: invalid syntax (4148340436.py, line 1)

In [24]:
liquidity

416659080672654